In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

# restore saved variables
%store -r summary_drug2_ky_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['drug_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['drug_two_year'].values

In [3]:
#### CART
depth = [7,8,9]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.5]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [4]:
cart_summary['FL_score'], ebm_summary['FL_score']

(0.623550405576365, 0.6203023838464019)

### Lasso Stumps

In [16]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['drug_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['drug_two_year'].values

## columns 
cols = KY_X.columns

In [25]:
c_grid={'C': [0.001, 0.002]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [26]:
stumps_summary['FL_score'], stumps_summary['dictionary']

(0.6480834346240667,
 {'age_at_current_charge28.0': 0.003979099836078727,
  'age_at_current_charge30.0': 0.002282686774506707,
  'age_at_current_charge31.0': 0.006344233341063798,
  'age_at_current_charge34.0': 0.06401965814279668,
  'age_at_current_charge35.0': 0.037724403451090946,
  'age_at_current_charge39.0': 0.06945071452406233,
  'age_at_current_charge45.0': 0.006254494079733146,
  'age_at_current_charge46.0': 0.019148942621788182,
  'age_at_current_charge70.0': -0.3222067408779261,
  'p_arrest1': -0.2543678234854165,
  'p_arrest2': 0.40286524673243,
  'p_arrest3': 0.20634639361947743,
  'p_arrest4': 0.08657936032538548,
  'p_arrest5': 0.06283313623901908,
  'p_arrest6': 0.04439691360628563,
  'p_charges1': -0.18878247197420092,
  'p_felony1': 0.028101687283429714,
  'p_misdemeanor3': 0.03633855134707658,
  'p_traffic1': -0.056232119377196695,
  'p_drug1': 0.5060016017710582,
  'p_drug2': 0.058855038110921895,
  'p_stealing1': -0.1868709439805156,
  'p_pending_charge1': 0.393309

### RiskSLIM

In [27]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['drug_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['drug_two_year'].values

## columns 
cols = KY_X.columns

In [28]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.001, random_state=816).fit(KY_X, KY_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

15

In [29]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [33]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'drug_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000, 
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/21/19 @ 12:17 PM | 203 rows in lookup table
10/21/19 @ 12:17 PM | ------------------------------------------------------------
10/21/19 @ 12:17 PM | runnning initialization procedure
10/21/19 @ 12:17 PM | ------------------------------------------------------------
10/21/19 @ 12:17 PM | CPA produced 2 cuts
10/21/19 @ 12:17 PM | running naive rounding on 5 solutions
10/21/19 @ 12:17 PM | best objective value: 0.6931
10/21/19 @ 12:17 PM | rounding produced 5 integer solutions
10/21/19 @ 12:17 PM | best objective value is 0.6931
10/21/19 @ 12:17 PM | running sequential rounding on 5 solutions
10/21/19 @ 12:17 PM | best objective value: 0.6931
10/21/19 @ 12:17 PM | sequential rounding produced 0 integer solutions
10/21/19 @ 12:17 PM | polishing 5 solutions
10/21/19 @ 12:17 PM | best objective value: 0.6931
10/21/19 @ 12:17 PM | polishing produced 5 integer solutions
10/21/19 @ 12:17 PM | initialization produced 9 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 12:17 PM | adding 206 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3143.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 12:36 PM | adding 216 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3127.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 12:37 PM | adding 229 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3142.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 12:39 PM | adding 212 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3146.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 12:40 PM | adding 206 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3159.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 12:42 PM | adding 227 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3155.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [35]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_train_score'])

(0.6314448813037978, 0.641709489695672)

In [36]:
#### save results
summary_drug2_ky_inter_model = {"CART": cart_summary,
                                "EBM": ebm_summary,
                                "Stumps": stumps_summary,
                                "RiskSLIM": riskslim_summary}
%store summary_drug2_ky_inter_model

Stored 'summary_drug2_ky_inter_model' (dict)


In [37]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Two Year\\"
results = [["Drug",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Two Year KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)